In [31]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import r2_score, mean_squared_error,median_absolute_error
from sklearn.preprocessing import MinMaxScaler
from IPython.core.interactiveshell import InteractiveShell
from datetime import timedelta
InteractiveShell.ast_node_interactivity = "all"
%matplotlib inline
sns.set()
pd.options.display.max_rows = 100

df_before = pd.read_csv('CR_FC_PREP_0522.csv')
df_before['POD'] = pd.to_datetime(df_before['POD'])

df_before.head()
df_before.tail()
df_before.info()
print(f'Dataset size: {df_before.shape}')

# Y is the target
Y = df_before.pop('RATE') 
X = df_before

#normalise
sc = StandardScaler()
X = sc.fit_transform(X)
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.3)

,POD,RATE
0,2005-07-19,2200.0
1,2005-07-20,2250.0
2,2005-07-21,2300.0
3,2005-07-22,2280.0
4,2005-07-23,2260.0


,POD,RATE
6370,2022-12-27,4121.43
6371,2022-12-28,4142.86
6372,2022-12-29,4164.29
6373,2022-12-30,4185.71
6374,2022-12-31,4207.14


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6375 entries, 0 to 6374
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype         
---  ------  --------------  -----         
 0   POD     6375 non-null   datetime64[ns]
 1   RATE    6375 non-null   float64       
dtypes: datetime64[ns](1), float64(1)
memory usage: 99.7 KB
Dataset size: (6375, 2)


In [32]:
# After 2023 data to compare accuracy of forecast
df_after = pd.read_csv('CR_FC_PREP_23.csv')
df_after['POD'] = pd.to_datetime(df_after['POD'])

# df_after.set_index('POD',inplace=True)

df_after.head()
df_after.tail()
df_after.info()
print(f'Dataset size: {df_after.shape}')


actual_X = df_after.drop("RATE", axis=1)
actual_y = df_after["RATE"]

,POD,RATE
0,2023-01-01,4228.57
1,2023-01-02,4250.00
2,2023-01-03,2800.00
3,2023-01-04,2800.00
4,2023-01-05,2800.00


,POD,RATE
47,2023-02-17,1500.0
48,2023-02-18,1500.0
49,2023-02-19,1500.0
50,2023-02-20,1500.0
51,2023-02-21,1500.0


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 52 entries, 0 to 51
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype         
---  ------  --------------  -----         
 0   POD     52 non-null     datetime64[ns]
 1   RATE    52 non-null     float64       
dtypes: datetime64[ns](1), float64(1)
memory usage: 960.0 bytes
Dataset size: (52, 2)


In [33]:
def findRF(X_train, X_test, y_train, y_test):
    # Build the random forest model
    rf_model = RandomForestRegressor()
    rf_model.fit(X_train, y_train)

    # Predict the test data
    rf_y_pred = rf_model.predict(X_test)

    # Calculate the root mean squared error
    rmse = mean_squared_error(y_test, rf_y_pred, squared=False)
    
    # Calculate the R-squared value
    r2 = r2_score(y_test, rf_y_pred)

    # Calculate the accuracy score in percentage
    mean = y_test.mean()
    accuracy = (1 - rmse/mean) * 100

    print("Root Mean Squared Error:", round(rmse, 3))
    print("R-squared Value:", round(r2, 3))
    print("Accuracy Score:", round(accuracy, 2), "%")

    return rf_y_pred

In [34]:
def findDT(X_train, X_test, y_train, y_test):
    # Train the model
    model = DecisionTreeRegressor()
    model.fit(X_train, y_train)

    # Make predictions on test data
    dt_y_pred = model.predict(X_test)

    # Calculate mean squared error between actual and predicted values
    mse = mean_squared_error(y_test, dt_y_pred)

    # Calculate root mean squared error
    rmse = np.sqrt(mse)

    # Calculate R-squared value
    r2 = r2_score(y_test, dt_y_pred)

    # Calculate mean value of your actual data
    mean = np.mean(y_test)

    # Calculate accuracy of your model in percentage
    accuracy = (1 - rmse/mean) * 100

    print('Root Mean Squared Error:', round(rmse, 3))
    print('R-squared:', round(r2, 3))
    print(f"Accuracy: {accuracy:.2f}%")

    return dt_y_pred

In [35]:
def findACC(y_pred, actual_y):
    # Calculate the accuracy of the model using mean absolute error and R-squared
   mae = median_absolute_error(y_pred, actual_y)
   r2 = r2_score(y_pred, actual_y)

   print(f"Mean absolute error: {mae:.4f}")
   print(f"R-squared value: {r2:.4f}")

In [36]:
n_periods = 52

# Random Forest
print("Random Forest Regressor")
rf_y_pred = findRF(X_train, X_test, y_train, y_test)[-n_periods:]
print(rf_y_pred)
print("\n")

# Decision Tree
print("Decision Tree Regressor:")
dt_y_pred = findRF(X_train, X_test, y_train, y_test)[-n_periods:]
print(dt_y_pred)
print("\n")

Random Forest Regressor
Root Mean Squared Error: 77.61
R-squared Value: 0.999
Accuracy Score: 95.29 %
[ 448.6647  723.975   500.      800.      738.7127  956.533  1203.
 6952.0021  749.8933  748.117   888.7121 1520.      400.     1300.
  750.      881.66   1144.6675  475.18    900.     8845.835   900.
  993.6746  770.      252.7     420.      450.     1500.     1636.1667
  400.      782.51   8846.7     779.3     411.2571  595.      420.
  455.2229 1193.5118  400.      652.7145 2000.      900.      401.28
  625.      566.4187  821.33   1032.1177 8884.4645 1311.1282  266.42
 1200.     1269.     9600.    ]


Decision Tree Regressor:
Root Mean Squared Error: 77.945
R-squared Value: 0.999
Accuracy Score: 95.27 %
[ 450.2644  724.15    500.      800.      738.5697  957.6478 1202.5
 6931.4311  741.9882  748.9624  889.1758 1520.      400.     1300.
  750.      875.15   1136.0007  475.      900.     9017.5121  900.
  994.4328  770.      250.      420.3     450.     1500.     1646.9332
  400.    

In [37]:
# Random Forest
print("Random forest regressor forecast vs actual cost rate results:")
findACC(rf_y_pred, actual_y)
print("\n")

# Decision Tree
print("Decision tree regressor forecast vs actual cost rate results:")
findACC(dt_y_pred, actual_y)
print("\n")


Random forest regressor forecast vs actual cost rate results:
Mean absolute error: 1820.3200
R-squared value: -0.3270


Decision tree regressor forecast vs actual cost rate results:
Mean absolute error: 1820.3200
R-squared value: -0.3235




In [38]:
# # Plot the forecast with current dataset
# plt.figure(figsize=(30, 8))
# plt.plot(df_bf_indexed)
# plt.plot(forecast_df, color='darkgreen')
# plt.title("Final Forecast of COST_RATE")
# plt.show()